In [1]:
import pandas as pd

In [3]:
data_df = pd.read_csv("../data/baidu_sentence.csv")

In [4]:
data_df.head()

,labels,content,subject
0,高中 生物 分子与细胞 组成细胞的化学元素 组成细胞的化合物,菠菜 土壤 中 吸收 氮 元素 用来 合成 淀粉 纤维素 葡萄糖 核酸 蛋白质 麦芽糖 脂肪酸,生物
1,高中 生物 稳态与环境 神经调节和体液调节的比较,下列 生物体 内 信息 传递 叙述 正确 下丘脑 分泌 促 甲状腺 激素 释放 激素 作用 ...,生物
2,高中 生物 生物技术实践 生物工程技术,自然 菌样 筛选 理想 生产 菌种 步骤 采集 菌样 富集 培养 纯种 分离 性能 测定 不...,生物
3,高中 生物 生物技术实践 生物技术在其他方面的应用 器官移植 复等位基因 胚胎移植 基因工程...,目前 精子 载体 法 逐渐 成为 具有 诱惑力 制备 转基因 动物 方法 方法 精子 外源 ...,生物
4,高中 地理 宇宙中的地球 地球运动的地理意义,某人 想 普通 飞机 一年 中 连续 两次 生日 认为 应 穿越 赤道 两级 本初子午线 国...,地理


In [17]:
import re
import jieba
def clean_sentence(line):
    line = re.sub(
            "[a-zA-Z0-9]|[\s+\-\|\!\/\[\]\{\}_,.$%^*(+\"\')]+|[:：+——()?【】《》“”！，。？、~@#￥%……&*（）]+|题目", '',line)
    words = jieba.cut(line, cut_all=False)
    
    return words

In [18]:
stopwords_path = "../data/stopwords/哈工大停用词表.txt"
def load_stop_words(stopwords_path):
    with open(stopwords_path, 'r') as f:
        stop_words = [word.strip() for word in f.readlines()]
        stop_words_dict = dict(zip(stop_words, list(range(len(stop_words)))))
    
    return stop_words_dict

In [19]:
def sentence_proc(sentence):
    # 清除无用词
    cw_words = list(clean_sentence(sentence))
    # print("after clear sentence:", len(list(cw_words)))
    
    # 去除停用词
    sw_words = [word for word in cw_words if word not in load_stop_words(stopwords_path)]
    # print("after clear stop_words:", len(sw_words))
    
    return " ".join(sw_words)

In [20]:
%%time
data_df["labels"] = data_df["labels"].apply(sentence_proc)

Building prefix dict from the default dictionary ...
Loading model from cache /tmp/jieba.cache
Loading model cost 0.543 seconds.
Prefix dict has been built successfully.


CPU times: user 1min 29s, sys: 5.98 s, total: 1min 35s
Wall time: 1min 35s


In [21]:
data_df["merged"] = data_df[["labels", "content"]].apply(lambda x: ' '.join(x), axis=1)

In [22]:
data_df.head()

,labels,content,subject,merged
0,高中生物 分子 细胞 组成 细胞 化学元素 组成 细胞 化合物,菠菜 土壤 中 吸收 氮 元素 用来 合成 淀粉 纤维素 葡萄糖 核酸 蛋白质 麦芽糖 脂肪酸,生物,高中生物 分子 细胞 组成 细胞 化学元素 组成 细胞 化合物 菠菜 土壤 中 吸收 氮 元...
1,高中生物 稳态 环境 神经 调节 体液 调节 比较,下列 生物体 内 信息 传递 叙述 正确 下丘脑 分泌 促 甲状腺 激素 释放 激素 作用 ...,生物,高中生物 稳态 环境 神经 调节 体液 调节 比较 下列 生物体 内 信息 传递 叙述 正确...
2,高中生物 生物 技术 实践 生物 工程技术,自然 菌样 筛选 理想 生产 菌种 步骤 采集 菌样 富集 培养 纯种 分离 性能 测定 不...,生物,高中生物 生物 技术 实践 生物 工程技术 自然 菌样 筛选 理想 生产 菌种 步骤 采集 ...
3,高中生物 生物 技术 实践 生物 技术 方面 应用 器官移植 复 等位基因 胚胎 移植 基因...,目前 精子 载体 法 逐渐 成为 具有 诱惑力 制备 转基因 动物 方法 方法 精子 外源 ...,生物,高中生物 生物 技术 实践 生物 技术 方面 应用 器官移植 复 等位基因 胚胎 移植 基因...
4,高中地理 宇宙 中 地球 地球 运动 地理 意义,某人 想 普通 飞机 一年 中 连续 两次 生日 认为 应 穿越 赤道 两级 本初子午线 国...,地理,高中地理 宇宙 中 地球 地球 运动 地理 意义 某人 想 普通 飞机 一年 中 连续 两次...


In [23]:
data_df.to_csv("../data/merged_labels_content.csv", index=None, header=True)

In [42]:
data_df["merged"].to_csv("../data/merged_labels_content_no_header.csv", index=None, header=False)

## 训练word2vec

In [43]:
merged_df = pd.read_csv("../data/merged_labels_content_no_header.csv")

In [44]:
merged_df.head()

,高中生物 分子 细胞 组成 细胞 化学元素 组成 细胞 化合物 菠菜 土壤 中 吸收 氮 元素 用来 合成 淀粉 纤维素 葡萄糖 核酸 蛋白质 麦芽糖 脂肪酸
0,高中生物 稳态 环境 神经 调节 体液 调节 比较 下列 生物体 内 信息 传递 叙述 正确...
1,高中生物 生物 技术 实践 生物 工程技术 自然 菌样 筛选 理想 生产 菌种 步骤 采集 ...
2,高中生物 生物 技术 实践 生物 技术 方面 应用 器官移植 复 等位基因 胚胎 移植 基因...
3,高中地理 宇宙 中 地球 地球 运动 地理 意义 某人 想 普通 飞机 一年 中 连续 两次...
4,高中地理 宇宙 中 地球 太阳 地球 影响 太阳辐射 地球 影响 叙述 正确 太阳 放出 能...


In [25]:
from gensim.models.word2vec import LineSentence
from gensim.models import word2vec
import gensim

import logging

In [26]:
logging.basicConfig(format="%(asctime)s : %(levelname)s : %(message)s", level=logging.INFO)

In [53]:
sentences = word2vec.LineSentence("../data/merged_labels_content_no_header.csv")

In [55]:
list(sentences)[:4]

[['高中生物',
  '分子',
  '细胞',
  '组成',
  '细胞',
  '化学元素',
  '组成',
  '细胞',
  '化合物',
  '菠菜',
  '土壤',
  '中',
  '吸收',
  '氮',
  '元素',
  '用来',
  '合成',
  '淀粉',
  '纤维素',
  '葡萄糖',
  '核酸',
  '蛋白质',
  '麦芽糖',
  '脂肪酸'],
 ['高中生物',
  '稳态',
  '环境',
  '神经',
  '调节',
  '体液',
  '调节',
  '比较',
  '下列',
  '生物体',
  '内',
  '信息',
  '传递',
  '叙述',
  '正确',
  '下丘脑',
  '分泌',
  '促',
  '甲状腺',
  '激素',
  '释放',
  '激素',
  '作用',
  '甲状腺',
  '小肠',
  '黏膜',
  '产生',
  '促',
  '胰液',
  '素可',
  '作用',
  '胰岛',
  '细胞',
  '突触',
  '前膜',
  '释放',
  '神经递质',
  '作用',
  '肌肉',
  '腺体',
  '燕麦',
  '幼根',
  '细胞分裂',
  '素',
  '含量',
  '高',
  '促进',
  '乙烯',
  '合成'],
 ['高中生物',
  '生物',
  '技术',
  '实践',
  '生物',
  '工程技术',
  '自然',
  '菌样',
  '筛选',
  '理想',
  '生产',
  '菌种',
  '步骤',
  '采集',
  '菌样',
  '富集',
  '培养',
  '纯种',
  '分离',
  '性能',
  '测定',
  '不同',
  '微生物',
  '生存环境',
  '不同',
  '获得理想',
  '微生物',
  '第一步',
  '是从',
  '适合',
  '环境',
  '采集',
  '菌样',
  '再',
  '一定',
  '方法',
  '分离',
  '纯化',
  '培养',
  '嗜',
  '盐菌',
  '菌样',
  '应从',
  '环境',
  '采集',
  '富集',
  '培养',
  '指',
  '创设',
 

In [54]:
len(list(sentences))

22576

In [35]:
help(gensim.models.Word2Vec)

Help on class Word2Vec in module gensim.models.word2vec:

class Word2Vec(gensim.models.base_any2vec.BaseWordEmbeddingsModel)
 |  Train, use and evaluate neural networks described in https://code.google.com/p/word2vec/.
 |  
 |  Once you're finished training a model (=no more updates, only querying)
 |  store and use only the :class:`~gensim.models.keyedvectors.KeyedVectors` instance in `self.wv` to reduce memory.
 |  
 |  The model can be stored/loaded via its :meth:`~gensim.models.word2vec.Word2Vec.save` and
 |  :meth:`~gensim.models.word2vec.Word2Vec.load` methods.
 |  
 |  The trained word vectors can also be stored/loaded from a format compatible with the
 |  original word2vec implementation via `self.wv.save_word2vec_format`
 |  and :meth:`gensim.models.keyedvectors.KeyedVectors.load_word2vec_format`.
 |  
 |  Some important attributes are the following:
 |  
 |  Attributes
 |  ----------
 |  wv : :class:`~gensim.models.keyedvectors.Word2VecKeyedVectors`
 |      This object essent

In [52]:
wv_model = word2vec.Word2Vec(LineSentence("../data/merged_labels_content_no_header.csv"), workers=8, min_count=5, size=200, iter=8)

2020-04-29 02:49:04,197 : INFO : collecting all words and their counts
2020-04-29 02:49:04,199 : INFO : PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2020-04-29 02:49:04,345 : INFO : PROGRESS: at sentence #10000, processed 575860 words, keeping 31793 word types
2020-04-29 02:49:04,483 : INFO : PROGRESS: at sentence #20000, processed 1156236 words, keeping 44053 word types
2020-04-29 02:49:04,519 : INFO : collected 46424 word types from a corpus of 1305480 raw words and 22576 sentences
2020-04-29 02:49:04,520 : INFO : Loading a fresh vocabulary
2020-04-29 02:49:04,546 : INFO : effective_min_count=5 retains 14244 unique words (30% of original 46424, drops 32180)
2020-04-29 02:49:04,547 : INFO : effective_min_count=5 leaves 1249050 word corpus (95% of original 1305480, drops 56430)
2020-04-29 02:49:04,577 : INFO : deleting the raw counts dictionary of 46424 items
2020-04-29 02:49:04,579 : INFO : sample=0.001 downsamples 44 most-common words
2020-04-29 02:49:04,579 : IN

2020-04-29 02:49:14,436 : INFO : EPOCH 8 - PROGRESS: at 74.94% examples, 845212 words/s, in_qsize 8, out_qsize 2
2020-04-29 02:49:14,644 : INFO : worker thread finished; awaiting finish of 7 more threads
2020-04-29 02:49:14,647 : INFO : worker thread finished; awaiting finish of 6 more threads
2020-04-29 02:49:14,648 : INFO : worker thread finished; awaiting finish of 5 more threads
2020-04-29 02:49:14,649 : INFO : worker thread finished; awaiting finish of 4 more threads
2020-04-29 02:49:14,661 : INFO : worker thread finished; awaiting finish of 3 more threads
2020-04-29 02:49:14,664 : INFO : worker thread finished; awaiting finish of 2 more threads
2020-04-29 02:49:14,665 : INFO : worker thread finished; awaiting finish of 1 more threads
2020-04-29 02:49:14,668 : INFO : worker thread finished; awaiting finish of 0 more threads
2020-04-29 02:49:14,669 : INFO : EPOCH - 8 : training on 1305480 raw words (1134760 effective words) took 1.2s, 918114 effective words/s
2020-04-29 02:49:14,67

In [58]:
wv_model.wv.most_similar(['生物'], topn=10)

[('动物', 0.571663498878479),
 ('生物体', 0.5686122179031372),
 ('动植物', 0.5393213629722595),
 ('原核', 0.5141187906265259),
 ('植物', 0.5091243982315063),
 ('遗传物质', 0.5004448890686035),
 ('动物细胞', 0.49553942680358887),
 ('真核细胞', 0.48394379019737244),
 ('酵母菌', 0.476127952337265),
 ('细菌', 0.47345465421676636)]

In [59]:
wv_model.save("../data/word2vec.model")

2020-04-29 02:51:44,312 : INFO : saving Word2Vec object under ../data/word2vec.model, separately None
2020-04-29 02:51:44,314 : INFO : not storing attribute vectors_norm
2020-04-29 02:51:44,316 : INFO : not storing attribute cum_table
2020-04-29 02:51:44,590 : INFO : saved ../data/word2vec.model


In [61]:
len(wv_model.wv.vocab)

14244

In [62]:
wv_model.wv.save_word2vec_format("../data/word2vec" + ".bin", binary=True)

2020-04-29 02:56:42,395 : INFO : storing 14244x200 projection weights into ../data/word2vec.bin
